In [1]:
import pandas as pd # pandas dataframes
from sklearn.ensemble import RandomForestClassifier # model that takes in data to train and classify the imports
from sklearn.metrics import f1_score, make_scorer, classification_report 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # one hot encoding of team names
import numpy as np
from collections import defaultdict
import time


In [2]:
# import csv files from git repository
teamPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/teamData.csv'
playerPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/outcomeData.csv'

In [3]:
# convert csv files into pandas dataframes
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

### Format All Data

In [4]:
# drop unwanted columns
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)

# change OT to integer value for number of OT rounds
outcomeData.loc[(outcomeData["OT?"].isna()),'OT?']= 0
outcomeData.loc[(outcomeData["OT?"] == "OT"),'OT?']= 1
outcomeData.loc[(outcomeData["OT?"] == "2OT"),'OT?']= 2
outcomeData.loc[(outcomeData["OT?"] == "3OT"),'OT?']= 3
outcomeData.loc[(outcomeData["OT?"] == "4OT"),'OT?']= 4


In [5]:
# function for redefining team names
def redefineTeamNames(dataframe):
    
    dataframe = dataframe.replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    
    dataframe = dataframe.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)

    dataframe = dataframe.replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    
    dataframe = dataframe.replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    
    dataframe = dataframe.replace('New Jersey Nets', 'Brooklyn Nets', regex=True)

    return dataframe

In [6]:
# call function to redefine team names
outcomeData = redefineTeamNames(outcomeData)
teamData = redefineTeamNames(teamData)

# set up encoder and use one hot encoding to transform team names
encoding = LabelEncoder()
encoding.fit(outcomeData['Home'].values)
outcomeData['HomeEncoded'] = encoding.transform(outcomeData['Home'].values)
outcomeData['VisitorEncoded'] = encoding.transform(outcomeData['Visitor'].values)

In [ ]:
# generate column names
teamDataColumn = teamData.drop(['Team', "Year", "Arena", "Attend.", "Attend./G", "G_PG", "MP_PG"], axis=1).columns
print(teamDataColumn)
testTeamDataColumn = teamData.drop(teamData.columns.difference(["FGA_PG","3PA_PG","3P_PG"]), axis=1).columns
print(testTeamDataColumn)
outcomeData = outcomeData.loc[outcomeData['Year'] == 2020]
outcomeData

### Feature Engineering

#### y data

In [8]:
# function for finding if the home team is the winner
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False

In [9]:
# add in HomeWin column
outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)

In [10]:
# spread
outcomeData["Final Home Spread"] = (outcomeData["VisitorPTS"] - outcomeData["HomePTS"]).astype(float)

#### x data

In [11]:
def differentialVariableGenerator(outcomeData, teamData, teamDataColumn):
                                  
    global teamData_global
    global counter 
    global differentialFeatureData
    
    differentialFeatureData = pd.DataFrame(index=range(outcomeData.shape[0]), columns = teamDataColumn)
    print(outcomeData.shape[0])
    teamData_global = teamData
    totalCount = outcomeData.shape[0]
    counter = 0
    overallTime = time.time()
    
    def calcDifferences(row):
                                  
        global teamData_global
        global counter
                                  
        counter = counter + 1;
        start = time.time()
        year = row['Year']
        visitor = row['Visitor']
        home = row['Home']
                                  
        def columnCalc(col):
                                  
            variable = col.name
            year_subset = teamData_global.loc[(teamData_global.Year == year)]
            visitorValue = year_subset.loc[year_subset.Team.astype(str).str.contains(visitor)][variable]
            homeValue = year_subset.loc[year_subset.Team.astype(str).str.contains(home)][variable]
            difference = homeValue.iloc[0] - visitorValue.iloc[0]
            differentialFeatureData[variable][counter-1] = difference
            #print(homeValue, visitorValue, difference)
                                  
        teamData_global[teamDataColumn].apply(columnCalc)
        print(visitor, home, time.time() - start,time.time() - overallTime)
        print("{count}/{total}".format(count = counter, total = totalCount))
                                  
    outcomeData.apply(calcDifferences, axis=1)
    print("Total time {time} seconds".format(time = time.time() - overallTime))
    return differentialFeatureData

In [ ]:
testOutcomeData = differentialVariableGenerator(outcomeData, teamData, teamDataColumn)

971
463    40.6
Name: FG_PG, dtype: float64 455    42.6
Name: FG_PG, dtype: float64 -2.0
463    88.5
Name: FGA_PG, dtype: float64 455    92.2
Name: FGA_PG, dtype: float64 -3.700000000000003
463    0.458
Name: FG%_PG, dtype: float64 455    0.462
Name: FG%_PG, dtype: float64 -0.004000000000000059
463    13.8
Name: 3P_PG, dtype: float64 455    14.0
Name: 3P_PG, dtype: float64 -0.1999999999999993
463    37.0
Name: 3PA_PG, dtype: float64 455    37.6
Name: 3PA_PG, dtype: float64 -0.6000000000000014
463    0.371
Name: 3P%_PG, dtype: float64 455    0.372
Name: 3P%_PG, dtype: float64 -0.0010000000000000564
463    26.8
Name: 2P_PG, dtype: float64 455    28.6
Name: 2P_PG, dtype: float64 -1.8000000000000007
463    51.5
Name: 2PA_PG, dtype: float64 455    54.6
Name: 2PA_PG, dtype: float64 -3.1000000000000014
463    0.521
Name: 2P%_PG, dtype: float64 455    0.525
Name: 2P%_PG, dtype: float64 -0.0040000000000000036
463    18.1
Name: FT_PG, dtype: float64 455    16.9
Name: FT_PG, dtype: float64 1.2000

Name: TRB_OppPP, dtype: float64 455    43.0
Name: TRB_OppPP, dtype: float64 2.700000000000003
463    25.3
Name: AST_OppPP, dtype: float64 455    23.5
Name: AST_OppPP, dtype: float64 1.8000000000000007
463    7.0
Name: STL_OppPP, dtype: float64 455    8.2
Name: STL_OppPP, dtype: float64 -1.1999999999999993
463    5.2
Name: BLK_OppPP, dtype: float64 455    4.6
Name: BLK_OppPP, dtype: float64 0.6000000000000005
463    16.6
Name: TOV_OppPP, dtype: float64 455    13.8
Name: TOV_OppPP, dtype: float64 2.8000000000000007
463    19.7
Name: PF_OppPP, dtype: float64 455    19.6
Name: PF_OppPP, dtype: float64 0.09999999999999787
463    105.2
Name: PTS_OppPP, dtype: float64 455    111.6
Name: PTS_OppPP, dtype: float64 -6.3999999999999915
463    26.6
Name: Age, dtype: float64 455    25.4
Name: Age, dtype: float64 1.2000000000000028
463    46
Name: W, dtype: int64 455    28
Name: W, dtype: int64 18
463    18
Name: L, dtype: int64 455    36
Name: L, dtype: int64 -18
463    44
Name: PW, dtype: int64 45

Name: FTA_OppPP, dtype: float64 464    22.6
Name: FTA_OppPP, dtype: float64 1.6999999999999993
456    0.762
Name: FT%_OppPP, dtype: float64 464    0.774
Name: FT%_OppPP, dtype: float64 -0.01200000000000001
456    10.6
Name: ORB_OppPP, dtype: float64 464    9.7
Name: ORB_OppPP, dtype: float64 0.9000000000000004
456    33.8
Name: DRB_OppPP, dtype: float64 464    32.1
Name: DRB_OppPP, dtype: float64 1.6999999999999957
456    44.5
Name: TRB_OppPP, dtype: float64 464    41.8
Name: TRB_OppPP, dtype: float64 2.700000000000003
456    23.0
Name: AST_OppPP, dtype: float64 464    23.0
Name: AST_OppPP, dtype: float64 0.0
456    7.7
Name: STL_OppPP, dtype: float64 464    8.1
Name: STL_OppPP, dtype: float64 -0.39999999999999947
456    4.8
Name: BLK_OppPP, dtype: float64 464    3.7
Name: BLK_OppPP, dtype: float64 1.0999999999999996
456    13.9
Name: TOV_OppPP, dtype: float64 464    15.5
Name: TOV_OppPP, dtype: float64 -1.5999999999999996
456    22.3
Name: PF_OppPP, dtype: float64 464    21.2
Name: PF

479    14.0
Name: FT_OppPG, dtype: float64 476    19.7
Name: FT_OppPG, dtype: float64 -5.699999999999999
479    18.4
Name: FTA_OppPG, dtype: float64 476    26.0
Name: FTA_OppPG, dtype: float64 -7.600000000000001
479    0.758
Name: FT%_OppPG, dtype: float64 476    0.759
Name: FT%_OppPG, dtype: float64 -0.0010000000000000009
479    10.9
Name: ORB_OppPG, dtype: float64 476    10.2
Name: ORB_OppPG, dtype: float64 0.7000000000000011
479    35.0
Name: DRB_OppPG, dtype: float64 476    35.4
Name: DRB_OppPG, dtype: float64 -0.3999999999999986
479    46.0
Name: TRB_OppPG, dtype: float64 476    45.5
Name: TRB_OppPG, dtype: float64 0.5
479    26.6
Name: AST_OppPG, dtype: float64 476    25.4
Name: AST_OppPG, dtype: float64 1.2000000000000028
479    8.0
Name: STL_OppPG, dtype: float64 476    8.2
Name: STL_OppPG, dtype: float64 -0.1999999999999993
479    5.0
Name: BLK_OppPG, dtype: float64 476    5.9
Name: BLK_OppPG, dtype: float64 -0.9000000000000004
479    14.4
Name: TOV_OppPG, dtype: float64 476  

Name: TRB_PP, dtype: float64 472    42.4
Name: TRB_PP, dtype: float64 0.8000000000000043
468    26.2
Name: AST_PP, dtype: float64 472    24.5
Name: AST_PP, dtype: float64 1.6999999999999993
468    7.3
Name: STL_PP, dtype: float64 472    7.5
Name: STL_PP, dtype: float64 -0.20000000000000018
468    5.2
Name: BLK_PP, dtype: float64 472    4.6
Name: BLK_PP, dtype: float64 0.6000000000000005
468    13.2
Name: TOV_PP, dtype: float64 472    15.5
Name: TOV_PP, dtype: float64 -2.3000000000000007
468    19.8
Name: PF_PP, dtype: float64 472    20.0
Name: PF_PP, dtype: float64 -0.1999999999999993
468    110.3
Name: PTS_PP, dtype: float64 472    109.0
Name: PTS_PP, dtype: float64 1.2999999999999972
468    65
Name: G_OppPG, dtype: int64 472    66
Name: G_OppPG, dtype: int64 -1
468    241.5
Name: MP_OppPG, dtype: float64 472    241.9
Name: MP_OppPG, dtype: float64 -0.4000000000000057
468    39.5
Name: FG_OppPG, dtype: float64 472    41.9
Name: FG_OppPG, dtype: float64 -2.3999999999999986
468    88.1


...

Name: DRB_PG, dtype: float64 458    36.3
Name: DRB_PG, dtype: float64 -0.6999999999999957
466    44.4
Name: TRB_PG, dtype: float64 458    46.7
Name: TRB_PG, dtype: float64 -2.3000000000000043
466    24.5
Name: AST_PG, dtype: float64 458    27.0
Name: AST_PG, dtype: float64 -2.5
466    7.2
Name: STL_PG, dtype: float64 458    8.0
Name: STL_PG, dtype: float64 -0.7999999999999998
466    5.5
Name: BLK_PG, dtype: float64 458    5.6
Name: BLK_PG, dtype: float64 -0.09999999999999964
466    12.3
Name: TOV_PG, dtype: float64 458    15.3
Name: TOV_PG, dtype: float64 -3.0
466    19.2
Name: PF_PG, dtype: float64 458    20.8
Name: PF_PG, dtype: float64 -1.6000000000000014
466    113.2
Name: PTS_PG, dtype: float64 458    112.6
Name: PTS_PG, dtype: float64 0.6000000000000085
466    63
Name: G_PP, dtype: int64 458    65
Name: G_PP, dtype: int64 -2
466    15295
Name: MP_PP, dtype: int64 458    15625
Name: MP_PP, dtype: int64 -330
466    41.6
Name: FG_PP, dtype: float64 458    41.5
Name: FG_PP, dtype: fl

455    92.2
Name: FGA_PG, dtype: float64 452    90.7
Name: FGA_PG, dtype: float64 1.5
455    0.462
Name: FG%_PG, dtype: float64 452    0.454
Name: FG%_PG, dtype: float64 0.008000000000000063
455    14.0
Name: 3P_PG, dtype: float64 452    15.4
Name: 3P_PG, dtype: float64 -1.4000000000000004
455    37.6
Name: 3PA_PG, dtype: float64 452    44.3
Name: 3PA_PG, dtype: float64 -6.699999999999996
455    0.372
Name: 3P%_PG, dtype: float64 452    0.348
Name: 3P%_PG, dtype: float64 0.024000000000000077
455    28.6
Name: 2P_PG, dtype: float64 452    25.7
Name: 2P_PG, dtype: float64 2.900000000000002
455    54.6
Name: 2PA_PG, dtype: float64 452    46.4
Name: 2PA_PG, dtype: float64 8.200000000000003
455    0.525
Name: 2P%_PG, dtype: float64 452    0.554
Name: 2P%_PG, dtype: float64 -0.028999999999999915
455    16.9
Name: FT_PG, dtype: float64 452    20.5
Name: FT_PG, dtype: float64 -3.6000000000000014
455    23.2
Name: FTA_PG, dtype: float64 452    26.0
Name: FTA_PG, dtype: float64 -2.80000000000000

455    1.13
Name: SOS, dtype: float64 452    0.03
Name: SOS, dtype: float64 1.0999999999999999
455    0.3
Name: SRS, dtype: float64 452    3.78
Name: SRS, dtype: float64 -3.48
455    110.8
Name: ORtg, dtype: float64 452    113.8
Name: ORtg, dtype: float64 -3.0
455    111.6
Name: DRtg, dtype: float64 452    110.2
Name: DRtg, dtype: float64 1.3999999999999915
455   -0.8
Name: NRtg, dtype: float64 452    3.6
Name: NRtg, dtype: float64 -4.4
455    103.9
Name: Pace, dtype: float64 452    103.3
Name: Pace, dtype: float64 0.6000000000000085
455    0.252
Name: FTr, dtype: float64 452    0.287
Name: FTr, dtype: float64 -0.034999999999999976
455    0.408
Name: 3PAr, dtype: float64 452    0.488
Name: 3PAr, dtype: float64 -0.08000000000000002
455    0.567
Name: TS%, dtype: float64 452    0.578
Name: TS%, dtype: float64 -0.010999999999999899
455    0.538
Name: eFG%, dtype: float64 452    0.539
Name: eFG%, dtype: float64 -0.001000000000000112
455    13.7
Name: TOV%, dtype: float64 452    12.6
Name: 

478    41.7
Name: FG_OppPP, dtype: float64 469    40.9
Name: FG_OppPP, dtype: float64 0.8000000000000043
478    87.6
Name: FGA_OppPP, dtype: float64 469    90.5
Name: FGA_OppPP, dtype: float64 -2.9000000000000057
478    0.476
Name: FG%_OppPP, dtype: float64 469    0.452
Name: FG%_OppPP, dtype: float64 0.02400000000000002
478    13.4
Name: 3P_OppPP, dtype: float64 469    11.2
Name: 3P_OppPP, dtype: float64 2.200000000000001
478    34.4
Name: 3PA_OppPP, dtype: float64 469    32.0
Name: 3PA_OppPP, dtype: float64 2.3999999999999986
478    0.389
Name: 3P%_OppPP, dtype: float64 469    0.351
Name: 3P%_OppPP, dtype: float64 0.03799999999999998
478    28.4
Name: 2P_OppPP, dtype: float64 469    29.7
Name: 2P_OppPP, dtype: float64 -1.3000000000000007
478    53.2
Name: 2PA_OppPP, dtype: float64 469    58.5
Name: 2PA_OppPP, dtype: float64 -5.299999999999997
478    0.533
Name: 2P%_OppPP, dtype: float64 469    0.507
Name: 2P%_OppPP, dtype: float64 0.025999999999999912
478    16.9
Name: FT_OppPP, dtyp

Name: AST_OppPG, dtype: float64 463    25.6
Name: AST_OppPG, dtype: float64 -2.0
456    7.9
Name: STL_OppPG, dtype: float64 463    7.0
Name: STL_OppPG, dtype: float64 0.9000000000000004
456    4.9
Name: BLK_OppPG, dtype: float64 463    5.3
Name: BLK_OppPG, dtype: float64 -0.39999999999999947
456    14.2
Name: TOV_OppPG, dtype: float64 463    16.8
Name: TOV_OppPG, dtype: float64 -2.6000000000000014
456    22.8
Name: PF_OppPG, dtype: float64 463    20.0
Name: PF_OppPG, dtype: float64 2.8000000000000007
456    109.7
Name: PTS_OppPG, dtype: float64 463    106.5
Name: PTS_OppPG, dtype: float64 3.200000000000003
456    64
Name: G_OppPP, dtype: int64 463    64
Name: G_OppPP, dtype: int64 0
456    15435
Name: MP_OppPP, dtype: int64 463    15460
Name: MP_OppPP, dtype: int64 -25
456    38.5
Name: FG_OppPP, dtype: float64 463    37.5
Name: FG_OppPP, dtype: float64 1.0
456    88.2
Name: FGA_OppPP, dtype: float64 463    87.4
Name: FGA_OppPP, dtype: float64 0.7999999999999972
456    0.436
Name: FG%_

In [ ]:
testOutcomeData.to_csv('/Users/lorneez/projects/sports_predictor/Sports_Model/Data/feature_data.csv')
testOutcomeData

In [ ]:
x_columns = []
for col in teamDataColumn:
    x_columns.append(col + "_DIFF")
print(x_columns)

### Training Model

In [ ]:
# target column (in this case, whether or not the home team will win)
# y = outcomeData["HomeWin"]
y = outcomeData["Final Home Spread"]

# columns that will be used to make the prediction
X = outcomeData[x_columns]

# set up scorer for testing accuracy
scorer = make_scorer(f1_score, pos_label = None, average = "weighted")

In [ ]:
# Set up train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)

In [ ]:
# cross val score
clf = RandomForestClassifier(random_state = 14)
scores = cross_val_score(clf, X_train, y_train, scoring=scorer)

In [ ]:
# mean score
print(np.mean(scores))